In [12]:
from pymongo import MongoClient
import pickle
from pprint import pprint
import config
import pandas as pd
import function as func
import datetime
from typing import List, Tuple
from db_connect import EngineConnect as DatabaseConnect
from sqlalchemy.sql import text

In [13]:
class InitialSchemaModel:
    def __init__(
        self, 
        *kwargs,
        db: DatabaseConnect,
    ):
        self.db = db
            
    def dim_time(self):
        sql = """
            CREATE TABLE production_analytic.dim_time
            (
                id integer NOT NULL,
                time_actual time without time zone,
                hour_actual smallint,
                military_hour smallint,
                minute_actual integer,
                second_actual integer,
                minute_of_day integer,
                second_of_day integer,
                quarter_hour character varying COLLATE pg_catalog."default",
                am_pm character varying COLLATE pg_catalog."default",
                day_night character varying COLLATE pg_catalog."default",
                day_night_abbrev character varying COLLATE pg_catalog."default",
                time_period character varying COLLATE pg_catalog."default",
                time_period_abbrev character varying COLLATE pg_catalog."default",
                CONSTRAINT d_time_id_pk PRIMARY KEY (id)
            )
            WITH (
                OIDS = FALSE
            )
            TABLESPACE pg_default;

            ALTER TABLE production_analytic.dim_time
                OWNER to ps_database_statistic;

            GRANT ALL ON TABLE production_analytic.dim_time TO ps_database_statistic;

            GRANT ALL ON TABLE production_analytic.dim_time TO duynn_1;

            CREATE INDEX d_time_actual_idx
                ON production_analytic.dim_time USING btree
                (time_actual ASC NULLS LAST)
                TABLESPACE pg_default;
        """
    
    def dim_work_type(self):
        data = [
            { 'work_type_key': 1, 'name': 'In WorkShift', 'description': None },
            { 'work_type_key': 2, 'name': 'Overtime', 'description': None }
        ]
        return data 
    
    def run(self):
        self.db.update(self.dim_process(), config.DWH_ANALYTIC_SCHEMA, config.DWH_DIM_PROCESS_TABLE)
        self.db.update(self.dim_work_type(), config.DWH_ANALYTIC_SCHEMA, config.DWH_DIM_WORK_TYPE_TABLE)

In [14]:
class EclaimsModel:
    def dim_field(self):
        data = [
            {'field_name': 'claimNature', 'is_sub_field': True },
            {'field_name': 'diagnosisRemarks', 'is_sub_field': True },
            {'field_name': 'documentReasonCode', 'is_sub_field': True },
            {'field_name': 'gender', 'is_sub_field': False },
            {'field_name': 'patientName', 'is_sub_field': False },
            {'field_name': 'eventEndDate', 'is_sub_field': False },
            {'field_name': 'diagnosisCode', 'is_sub_field': False },
            {'field_name': 'documentName', 'is_sub_field': False },
            {'field_name': 'departmentType', 'is_sub_field': False },
            {'field_name': 'ageOrDOB', 'is_sub_field': False },
            {'field_name': 'diagnosisDescription', 'is_sub_field': False },
            {'field_name': 'departmentName', 'is_sub_field': False },
            {'field_name': 'eventDate', 'is_sub_field': False },
            {'field_name': 'hospitalName', 'is_sub_field': False },
            {'field_name': 'treatmentType', 'is_sub_field': False },
            {'field_name': 'birthDate', 'is_sub_field': False },
            {'field_name': 'isStamped', 'is_sub_field': False },
            {'field_name': 'hospitalCode', 'is_sub_field': False },
            {'field_name': 'treatmentDescription', 'is_sub_field': False },
            {'field_name': 'hospitalProvince', 'is_sub_field': False },
        ]
        return data
    
    def dim_remark(self):
        data = [
            {'remark_code': 'OCR-E01', 'remark_code_description': 'No Hospital Discharge Summary Exists' },
            {'remark_code': 'OCR-E05', 'remark_code_description': 'Multiple Discharge Summaries Exist' },
            {'remark_code': 'OCR-E03', 'remark_code_description': 'Others' },
            {'remark_code': 'OCR-E02', 'remark_code_description': 'Image is not clear' },
            {'remark_code': 'OCR-R01', 'remark_code_description': 'Handwritten discharge summary' },
            {'remark_code': 'OCR-E04', 'remark_code_description': 'Invoice - Revert back to MVL' },
        ]
        return data
  
    def run(self):
        self.db.update(self.dim_field(), config.DWH_ANALYTIC_SCHEMA, config.DWH_DIM_FIELD_ECLAIMS_TABLE)
        self.db.update(self.dim_remark(), config.DWH_ANALYTIC_SCHEMA, config.DWH_DIM_REMARK_ECLAIMS_TABLE)
        self.db.update(self.dim_working_time, config.DWH_ANALYTIC_SCHEMA, config.DWH_DIM_WORKING_TIME_ECLAIMS_TABLE)

In [16]:
db_connect = DatabaseConnect(uri = config.DWH_SQLALCHEMY_URI)
init_data = InitialDataModel(db = db_connect)
init_data.run()

init_data_eclaims = InitialDataEclaimsModel(db = db_connect)
init_data_eclaims.run()
db_connect.close()